In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
import pandas as pd

In [3]:
inference_server_url = "http://0.0.0.0:8000/v1"

chat = ChatOpenAI(
    model="models/llama3_lora_sft/",
    openai_api_key="None",
    openai_api_base=inference_server_url,
    temperature=0,
)

In [4]:
def pred(messages: str):
    messages = [
        SystemMessage(
            content="You are a helpful assistant who're always speak Thai."
        ),
        HumanMessage(
            content="จงสกัดคำสำคัญและวลีสำคัญจากข้อความจำนวน 5 คำ (หรือวลี) โดยตอบในรูปแบบ keyword1|keyword2|keyword3|keyword4|keyword5" + "\n" + messages
        ),
    ]
    return chat(messages).content

In [5]:
test = pd.read_csv("../train_lora/data/Test (1).csv")
test

,index,title,institute,description
0,0,การออกแบบสื่อสามมิติด้วยโปรแกรม Blender 3D | 3...,มหาวิทยาลัยราชภัฏเชียงราย,ศึกษาความรู้พื้นฐานงานออกแบบภาพกราฟิก 3 มิติเพ...
1,1,เทคนิคการถ่ายภาพในแบบสื่อสาธารณะ | Shooting te...,สถานีโทรทัศน์ไทยพีบีเอส (Thai PBS),ศึกษาเทคนิคการถ่ายภาพ การถ่ายวิดีโอในรูปแบบต่า...
2,2,ผู้ประกอบการร้านค้าออนไลน์ | e-Commerce for En...,วิทยาลัยเทคโนโลยีภาคใต้,NaN
3,3,ระบบราชการไทย | The Thai Civil Service System,สำนักงานคณะกรรมการข้าราชการพลเรือน,NaN
4,4,การจัดการทรัพยากรมนุษย์ | Human Resource Manag...,มหาวิทยาลัยนเรศวร,ทุกท่านจะได้เรียนรู้เกี่ยวกับมโนทัศน์การจัดการ...
...,...,...,...,...
173,173,การวิจัยด้านเทคโนโลยีสารสนเทศทางธุรกิจ | Resea...,มหาวิทยาลัยราชภัฏเชียงใหม่,NaN
174,174,โปรแกรมคอมพิวเตอร์และขั้นตอนวิธีทางคอมพิวเตอร์...,มหาวิทยาลัยขอนแก่น,ความรู้พื้นฐานเกี่ยวกับการทำงานของระบบคอมพิวเต...
175,175,การผลิตวรรณกรรมสำหรับเด็กและเยาวชน Generation ...,มหาวิทยาลัยขอนแก่น,ความหมาย ความสำคัญ ประเภทของวรรณกรรมสำหรับเด็ก...
176,176,อินเทอร์เน็ตของสรรพสิ่ง : จากแนวคิดไปสู่ผลิตภั...,มหาวิทยาลัยวลัยลักษณ์,อินเทอร์เน็ตของสรรพสิ่งเบื้องต้นและการประยุกต์...


In [27]:
from tqdm.auto import tqdm

bad_keywords = []
for idx, row in tqdm(test.iterrows(), total=len(test)):
    keyword = pred(row["text"])
    # print(row["text"]+"\n")
    # print(keyword)
    if len(keyword.split("|")) != 5:
        print("Bad keywords!")
        bad_keywords.append(idx)
    # print("=="*20)

    test.loc[idx, "keywords"] = keyword

  0%|          | 0/305 [00:00<?, ?it/s]

Bad keywords!


In [43]:
for bad_index in bad_keywords:
    keywords = pred(test.loc[bad_index, "text"])
    if len(keywords.split("|")) != 5:
        prompt = test.loc[bad_index, "text"] + "\n" + "keywords นี้ไม่ถูกต้อง จงแก้ไขใหม่:" + "\n" + keywords + "\n" + "keywords ที่ถูกต้อง:"
        new = pred(prompt)
        test.loc[bad_index, "keywords"] = new
    else:
        test.loc[bad_index, "keywords"] = keywords

In [29]:
test

,id,text,keywords
0,0,บทคัดย่อ การวิจัยเชิงพรรณนานี้มีวัตถุประสงค์เพ...,ความสิ้นหวัง|มุมมองเชิงเหตุผลด้านลบ|ความสามารถ...
1,1,การวิจัยนี้เป็นการวิจัยเชิงปริมาณ เพื่อศึกษาทั...,ทัศนคติทางดิจิทัล|ความเข้าใจทางดิจิทัล|ทักษะทา...
2,2,วัตถุประสงค์ เพื่อพัฒนาแบบคัดกรองสมองเสื่อมระด...,แบบคัดกรองสมองเสื่อม|สมองเสื่อม|ปฐมภูมิ|ทุติยภ...
3,3,การวิจัยเชิงพรรณนานี้ มีวัตถุประสงค์เพื่อศึกษา...,ความรู้|ทัศนคติ|ปัจจัยเอื้อ|ปัจจัยเสริม|พฤติกร...
4,4,วัตถุประสงค์ เพื่อพัฒนาฐานข้อมูลยาคู่เหมือนในป...,ยาคู่เหมือน|ฐานข้อมูล|เภสัชกรโรงพยาบาล|การค้นห...
...,...,...,...
300,300,การศึกษานี้เป็นการวิจัยเชิงพรรณนาแบบศึกษาย้อนห...,โรคติดเชื้อไวรัสโคโรนา|โรคติดเชื้อโคโรนา|โรคติ...
301,301,การวิจัยครั้งนี้เป็นการวิจัยเชิงพรรณนา มีวัตถุ...,แรงจูงใจในการทำงาน|ประชาคมอาเซียน|พยาบาลวิชาชี...
302,302,0.001 และ p วัตถุประสงค์ เพื่อเปรียบเทียบผลของ...,โรคจิตเภท|ความร่วมมือในการใช้ยา|ความรู้เกี่ยวก...
303,303,บทนำ เกมเป็นสื่ออิเลคโทรนิคที่สามารถเข้าถึงง่า...,การติดเกม|ภูมิคุ้มกันการติดเกม|เพศ|ความรู้|ทัศ...


In [48]:
test.drop("text", axis=1).to_csv("submission.csv", index=False)